In [1]:
import numpy as np
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def normalize(arr,cou):
  sum_elements=np.sum(arr)
  if(sum_elements!=0):
   mean=sum_elements/cou
   d=arr-mean
   std=np.sum(np.multiply(d,d))
   stdev=(std/cou)**0.5
   arr=(arr-mean)*(1/stdev)
  return arr

In [3]:
def get_data(temp):
 y_train=temp['1']
 x_train=temp
 del x_train['1']

 X_train=x_train.to_numpy()
 Y_train=y_train.to_numpy()
 r,c=X_train.shape
 #X_train=np.concatenate((np.ones((r,1)),X_train),axis=1)
 
 X_train=X_train.T
 for i in range(c):
  if(i==0):
   continue 
  X_train[i]=normalize(X_train[i],r)
 X_train=X_train.T
 
 return X_train,Y_train

In [87]:
temp= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emnist-letters-train (1).csv')
temp1=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emnist-letters-test.csv')

y1 = temp1['1']
Y1 = np.array(y1)
j = 5000
x,y=get_data(temp)
X = np.array(pd.DataFrame(x).head(j))
Y = np.array(pd.DataFrame(y).head(j))
x_train = X
y_train = Y

x1,y1=get_data(temp1)
X1 = np.array(pd.DataFrame(x1).head(j))
Y1 = np.array(pd.DataFrame(y1).head(j))
x_test = X1
y_test = Y1

In [7]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

In [12]:
class KMeans:
    def __init__(self, K=5, max_iters=100, plot_steps=False):
        self.K = K
        self.max_iters = max_iters
        self.plot_steps = plot_steps

        # list of sample indices for each cluster
        self.clusters = [[] for _ in range(self.K)]
        # the centers (mean feature vector) for each cluster
        self.centroids = []

    def predict(self, X):
        self.X = X
        self.n_samples, self.n_features = X.shape

        # initialize
        np.random.seed(42)
        random_sample_idxs = np.random.choice(self.n_samples, self.K, replace=False)
        self.centroids = [self.X[idx] for idx in random_sample_idxs]

        # Optimize clusters
        for _ in range(self.max_iters):
            # Assign samples to closest centroids (create clusters)
            self.clusters = self._create_clusters(self.centroids)
            # Calculate new centroids from the clusters
            centroids_old = self.centroids
            self.centroids = self._get_centroids(self.clusters)

            # check if clusters have changed
            if self._is_converged(centroids_old, self.centroids):
                break
        # Classify samples as the index of their clusters
        return self._get_cluster_labels(self.clusters)

    def _get_cluster_labels(self, clusters):
        # each sample will get the label of the cluster it was assigned to
        labels = np.empty(self.n_samples)

        for cluster_idx, cluster in enumerate(clusters):
            for sample_index in cluster:
                labels[sample_index] = cluster_idx
        return labels

    def _create_clusters(self, centroids):
        # Assign the samples to the closest centroids to create clusters
        clusters = [[] for _ in range(self.K)]
        for idx, sample in enumerate(self.X):
            centroid_idx = self._closest_centroid(sample, centroids)
            clusters[centroid_idx].append(idx)
        return clusters

    def _closest_centroid(self, sample, centroids):
        # distance of the current sample to each centroid
        distances = [euclidean_distance(sample, point) for point in centroids]
        closest_index = np.argmin(distances)
        return closest_index

    def _get_centroids(self, clusters):
        # assign mean value of clusters to centroids
        centroids = np.zeros((self.K, self.n_features))
        for cluster_idx, cluster in enumerate(clusters):
            cluster_mean = np.mean(self.X[cluster], axis=0)
            centroids[cluster_idx] = cluster_mean
        return centroids

    def _is_converged(self, centroids_old, centroids):
        # distances between each old and new centroids, fol all centroids
        distances = [
            euclidean_distance(centroids_old[i], centroids[i]) for i in range(self.K)
        ]
        return sum(distances) == 0

In [89]:
def accuracy(y_train,y_pred):
  success=0
  for i in range(y_train.shape[0]):
    if(y_pred[i]==y_train[i]):
      success+=1
  acc = 30*(success/y_train.shape[0])
  return acc

In [90]:
clusters = len(np.unique(y_train))
k = KMeans(K=clusters, max_iters=1000, plot_steps=True)
y_pred = (k.predict(x_test))
print(100*(accuracy(y_test,y_pred)))

84.6
